In [2]:
import pandas as pd
import numpy as np


In [3]:
#Laad de source dataset in een Pandas dataframe en zorg ervoor dat je op een zo efficient mogelijke manier enkel de voor deze dimensie relevant kolommen inleest


columns_to_use= ['host_id',
'host_name',
'host_since',
'host_location',
'host_response_time',
'host_acceptance_rate',
'host_is_superhost',
'host_listings_count']

df=pd.read_csv(r"C:\Users\mvdp1\OneDrive\Documenten\GitHub\Pandas-Oefeningen-Data\AirBNB Antwerpen - Listings.csv", usecols=columns_to_use)

In [ ]:
#Eerst en vooral gaan we kijken naar de dataframe wat er allemaal opgeschoont dient te worden

df

In [4]:
#Verwijder dubbele records je baserend op het host_id


df = df.drop_duplicates(subset=['host_id'])

In [5]:
#Splits de host_location kolom in een host_gemeente en een host_land kolom


split_location = df['host_location'].str.split(', ', expand=True)

df['host_gemeente'] = split_location[0]
df['host_land'] = split_location[1]

#Sommige host_location waarden hadden enkel een land als waarde, zorg ervoor dat deze landwaarde in de host_land kolom beland

df['host_land'] = df['host_land'].fillna(df['host_gemeente'])
df['host_gemeente'] = df['host_gemeente'].mask(df['host_gemeente'] == df['host_land'])

#Verwijder kolom host_location kolom, die hebben we niet meer nodig

df.drop(columns='host_location', inplace=True)

#Converteren van Amerikaanse staten initialen naar land USA

df.loc[df['host_land'].str.len() < 3, 'host_land']= 'USA'

In [6]:
#Aanpassen van kolom volgorde


df= df[['host_id','host_name','host_since','host_gemeente','host_land', 'host_response_time','host_acceptance_rate', 'host_is_superhost', 'host_listings_count']]	

In [7]:
#Converteer de kolommen naar het gepaste datatype 

df['host_since']= pd.to_datetime(df['host_since'])
df['host_acceptance_rate']=df['host_acceptance_rate'].str.replace('%',' ')
df['host_response_time']= df['host_response_time'].astype('category')
df['host_acceptance_rate']=df['host_acceptance_rate'].astype(float)
df['host_acceptance_rate']=df['host_acceptance_rate'].apply(lambda x: x/100)
df['host_is_superhost']=  df['host_is_superhost'].astype(bool)

In [8]:
#Pas kolom namen aan

df.rename(columns={'host_name': 'naam',
                   'host_since': 'host_sinds', 
                   'host_gemeente': 'woonplaats', 
                   'host_land': 'land',
                   'host_response_time':'reactietijd', 
                   'host_acceptance_rate': 'acceptatiegraad', 
                   'host_is_superhost': 'is_superhost', 
                   'host_listings_count': 'aantal_accomodaties' }, inplace=True)

In [9]:
#Zorg dat de string datavelden steeds in proper case staan


string_columns = df.select_dtypes(include='object').columns
df[string_columns] = df[string_columns].apply(lambda x: x.str.strip())
df[string_columns] = df[string_columns].apply(lambda x: x.str.capitalize())

In [10]:
#Verwijder niet relevante karakters, e.g. quotes en dubbele quotes uit de string datavelden


df[string_columns] = df[string_columns].apply(lambda x: x.str.replace('[\'"\;]', '', regex=True))

In [11]:
#Zorg dat de woonplaats waarden consistent zijn (in dit geval is Antwerpen in 2 talen geschreven wat data-analyse moeilijk maakt)


df['woonplaats']=df['woonplaats'].replace('Antwerp', 'Antwerpen')

In [12]:
#Voeg een extra ‘verblijft_in_het_buitenland’ boolean kolom toe


df['verblijft_in_het_buitenland'] = np.where(df['land'] != 'Belgium', True, False)

#Als de land kolom een NaN waarde bevat wordt de nieuwe kolom 'verblijft_in_het_buitenland' automatisch naar NaN omgezet


df.loc[df['land'].isna(), 'verblijft_in_het_buitenland'] = np.nan

In [13]:
#Zorg ervoor dat de host_id kolom als dataframe index wordt gebruikt

df.set_index('host_id', inplace=True)

In [18]:
#Nu even kijken naar het resultaat
df

,naam,host_sinds,woonplaats,land,reactietijd,acceptatiegraad,is_superhost,aantal_accomodaties,verblijft_in_het_buitenland
host_id,,,,,,,,,
234077,Karin,2010-09-14,Antwerpen,Belgium,within an hour,0.96,True,4,False
586942,Gina,2011-05-14,Antwerpen,Belgium,within an hour,0.97,True,1,False
1167377,Geert,2011-09-16,Antwerpen,Belgium,NaN,0.42,True,1,False
1263933,Kristien,2011-10-08,Antwerpen,Belgium,within an hour,1.00,True,1,False
1754396,Marleen,2012-02-15,Antwerpen,Belgium,within an hour,0.87,True,1,False
...,...,...,...,...,...,...,...,...,...
18992409,Veronique,2014-07-27,NaN,NaN,within an hour,1.00,True,1,NaN
382680622,Raf,2021-01-02,Brasschaat,Belgium,NaN,NaN,True,1,False
36131763,Lore,2015-06-18,Antwerpen,Belgium,within an hour,1.00,True,1,False


In [14]:
#Exporteer de opgeschoonde data naar een csv file

df.to_csv('AirBNB_data_cleaned.csv',index=False)